In [37]:
from linkedin_functions_t import *
import itertools
import pandas as pd

In [38]:
# location = ['Switzerland', 'Norway', 'Iceland', 'Hong Kong', 'Denmark', 'Sweden',
#             'Germany', 'Ireland', 'Singapore', 'Australia']

location = ['China', 'United States', 'United Kingdom', 'Singapore', 'Australia', 'Canada',
            'France', 'India', 'Japan', 'Germany', 'Malaysia', 'Hong Kong', 'Philippines']

schools_dict = {"The University of Hong Kong": ['The University of Hong Kong'],
                "The Chinese University of Hong Kong": ['The Chinese University of Hong Kong'],
                "The Hong Kong University of Science and Technology": ['The Hong Kong University of Science and Technology'],
                "The Hong Kong Polytechnic University": ['The Hong Kong Polytechnic University'],
                "City University of Hong Kong": ['City University of Hong Kong'],
                "National University of Singapore": ['National University of Singapore'],
                "Nanyang Technological University Singapore": ['Nanyang Technological University Singapore']}


# degrees_dict = {"Bachelor's Degree": ['Bachelors degree'],
#                 "Master's Degree": ['Masters degree'],
#                 'Undergraduate & Postgraduate': ['Bachelors degree', 'Masters degree', 'PhD']}


In [39]:
location_dict = {loc:[encodeFacet([loc], kind='locations')] for loc in location}

schools_dict = {k:[encodeFacet(schools, kind='schools')] for k, schools in schools_dict.items()}

# degrees_dict = {k:[encodeFacet(degrees, kind='degrees')] for k, degrees in degrees_dict.items()}

In [41]:
# segments = list(itertools.product(*[location_dict, schools_dict, degrees_dict]))

segments = list(itertools.product(*[location_dict, schools_dict]))

print(segments)


[('China', 'The University of Hong Kong'), ('China', 'The Chinese University of Hong Kong'), ('China', 'The Hong Kong University of Science and Technology'), ('China', 'The Hong Kong Polytechnic University'), ('China', 'City University of Hong Kong'), ('China', 'National University of Singapore'), ('China', 'Nanyang Technological University Singapore'), ('United States', 'The University of Hong Kong'), ('United States', 'The Chinese University of Hong Kong'), ('United States', 'The Hong Kong University of Science and Technology'), ('United States', 'The Hong Kong Polytechnic University'), ('United States', 'City University of Hong Kong'), ('United States', 'National University of Singapore'), ('United States', 'Nanyang Technological University Singapore'), ('United Kingdom', 'The University of Hong Kong'), ('United Kingdom', 'The Chinese University of Hong Kong'), ('United Kingdom', 'The Hong Kong University of Science and Technology'), ('United Kingdom', 'The Hong Kong Polytechnic Uni

In [42]:
%%time
columns = ['Country', 
           'School', 
           # 'Degree',
           'Count']
new_rows = []

for segment in segments:
    # unpack the tuple and get encodings
    # location_name, school_name, degree_name = segment
    location_name, school_name = segment
    # location, school, degree = location_dict[location_name], schools_dict[school_name], degrees_dict[degree_name]
    location, school = location_dict[location_name], schools_dict[school_name]
    
    # generate URL for request
    requestCriteria = createRequestDataForAudienceCounts(locations = location,
                                                         genders = [],
                                                         schools = school)
                                                         # degrees= degree)
                                                        
    
    # submit GET request
    count = getAudienceCounts(requestCriteria)
    
    # store data in a series (row)
    new_row = pd.Series(dtype=object)
    new_row['Country'] = location_name
    new_row['School'] = school_name
    # new_row['Degree'] = degree_name
    new_row['Count'] = count
    new_rows.append(new_row.values)
    
# construct dataframe
df = pd.DataFrame(new_rows, columns=columns)

CPU times: total: 516 ms
Wall time: 41.6 s


In [43]:
df

,Country,School,Count
0,China,The University of Hong Kong,12000
1,China,The Chinese University of Hong Kong,7300
2,China,The Hong Kong University of Science and Techno...,4300
3,China,The Hong Kong Polytechnic University,5000
4,China,City University of Hong Kong,3900
...,...,...,...
86,Philippines,The Hong Kong University of Science and Techno...,510
87,Philippines,The Hong Kong Polytechnic University,0
88,Philippines,City University of Hong Kong,0
89,Philippines,National University of Singapore,1500


In [44]:
df.to_excel('top10(no degree).xlsx', index=False)